# Trabalho Final de Inteligência Artificial
### Aluno: Paulo Henrique Dionysio | RA: 221026169

### Previsão de preços de casas usando redes neurais

## Importação de Bibliotecas Necessárias

In [18]:
# Importação das bibliotecas necessárias
import pandas as pd  # Manipulação de dados tabulares
import numpy as np  # Operações numéricas
import tensorflow as tf  # Construção e treinamento de redes neurais
from sklearn.preprocessing import MaxAbsScaler  # Normalização de dados
from sklearn.impute import SimpleImputer  # Tratamento de valores ausentes
from sklearn.model_selection import train_test_split  # Separação dos dados em treino e validação
import plotly.express as px  # Visualização de gráficos interativos


## Função para Carregar Dados

In [19]:
# Função para carregar e concatenar os dados de treino e teste
def carregar_dados(treino_file, teste_file):
    """
    Carrega os dados de treino e teste de arquivos CSV.
    Args:
        treino_file: Caminho para o arquivo de treino.
        teste_file: Caminho para o arquivo de teste.
    Returns:
        Tupla contendo (dados_treino, dados_teste, dados_concat).
    """
    dados_treino = pd.read_csv('train.csv')
    dados_teste = pd.read_csv('test.csv')
    dados = pd.concat([dados_treino, dados_teste], sort=False)
    return dados_treino, dados_teste, dados

## Função de Pré-Processamento

In [ ]:
# Função para realizar o pré-processamento dos dados
def preprocessar_dados(dados, dados_treino, scaler, imputer):
    """
    Pré-processa os dados de entrada, incluindo normalização e imputação.
    Args:
        dados: Dados combinados de treino e teste.
        dados_treino: Dados originais de treino.
        scaler: Objeto de normalização.
        imputer: Objeto para tratar valores ausentes.
    Returns:
        Dados pré-processados para treino, validação e teste.
    """
    # Transformação de variáveis categóricas em dummies
    cat_cols = ['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
                'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
                'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
                'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
                'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
                'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
                'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
                'GarageCond', 'PavedDrive', 'PoolQC', 'MiscFeature', 'SaleType',
                'SaleCondition', 'Fence'] # Variáveis categóricas
    juntos_num = pd.get_dummies(dados, columns=cat_cols) # Transformação em dummies
    
    # Separação dos dados em treino e teste
    treino_num = juntos_num.iloc[:len(dados_treino), :]
    teste_num = juntos_num.iloc[len(dados_treino):, :]

    X = treino_num.drop(["Id", "SalePrice"], axis=1)
    y = treino_num["SalePrice"]
    
    # Divisão dos dados de treino em treino e validação
    Xtr, Xval, ytr, yval = train_test_split(X, y, train_size=0.5, random_state=0)

    # Normalização e imputação
    Xtr = scaler.fit_transform(imputer.fit_transform(Xtr))
    Xval = scaler.transform(imputer.transform(Xval))
    ytr = ytr.values
    yval = yval.values

    return Xtr, Xval, ytr, yval, treino_num, teste_num


## Carregamento e Pré-Processamento

In [ ]:
# Configuração de caminhos para arquivos
treino_file = "caminho_para_train.csv"
teste_file = "caminho_para_test.csv"

# Carregamento dos dados
dados_treino, dados_teste, dados = carregar_dados(treino_file, teste_file)

# Configuração de objetos para normalização e imputação
scaler = MaxAbsScaler() # Normalização
imputer = SimpleImputer(strategy="median") # Imputação

# Pré-processamento
Xtr, Xval, ytr, yval, treino_num, teste_num = preprocessar_dados(
    dados, dados_treino, scaler, imputer
)


## Treinamento do Modelo

In [ ]:
# Configuração do modelo de rede neural
tf.random.set_seed(2) # Define a semente aleatória para reprodução dos resultados.

inp = tf.keras.layers.Input((Xtr.shape[1],)) # Define a camada de entrada da rede neural.
hid1 = tf.keras.layers.Dense(100, activation="relu")(inp) # Adiciona a primeira camada densa (fully connected). (cada neurônio recebe a entrada de todos os neurônios da camada anterior)
drop = tf.keras.layers.Dropout(0.5)(hid1) # Aplica o método de Dropout para reduzir o overfitting.
hid2 = tf.keras.layers.Dense(50, activation="relu")(drop) # Adiciona a segunda camada densa.
hid3 = tf.keras.layers.Dense(25, activation="relu")(hid2) # Adiciona a terceira camada densa.
out = tf.keras.layers.Dense(1, activation="relu")(hid3) # Adiciona a camada de saída.

mdl = tf.keras.Model(inp, out) # Define o modelo da rede neural.
mdl.compile(loss="mean_squared_logarithmic_error", optimizer="adam") # Compila o modelo.
es = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss", min_delta=0.1, patience=10, mode="min", restore_best_weights=True
) # Define o método de Early Stopping para evitar o overfitting.

# Treinamento
history = mdl.fit(Xtr, ytr, validation_data=(Xval, yval), epochs=100, shuffle=True, batch_size=1, callbacks=[es]) 


Epoch 1/100
730/730 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 60.7033 - val_loss: 7.7311
Epoch 2/100
730/730 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 5.4447 - val_loss: 1.4321
Epoch 3/100
730/730 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1.1131 - val_loss: 0.3187
Epoch 4/100
730/730 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.3374 - val_loss: 0.1256
Epoch 5/100
730/730 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1614 - val_loss: 0.1013
Epoch 6/100
730/730 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1535 - val_loss: 0.0949
Epoch 7/100
730/730 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1471 - val_loss: 0.0862
Epoch 8/100
730/730 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1271 - val_loss: 0.0770
Epoch 9/100
730/730 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1109 - val_loss: 0.0667
Epoch 10/100
730/730 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1034 - val_loss: 0.0595
Epoch 11/100
730/730 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0964 - val_loss: 0.0499
Epoch 12/100
730/730 ━━━━━━━━━━━━━━━━━━━

## Visualização dos Resultados

In [23]:
# Gráfico de perda durante o treinamento
fig1 = px.line(
    x=range(len(history.history["loss"])),
    y=[history.history["loss"], history.history["val_loss"]],
    labels={"x": "Épocas", "y": "Perda"},
    title="Perda de Treinamento vs Validação"
)
fig1.show()


## Previsões e Análise

In [24]:
# Geração de previsões
p_val = mdl.predict(Xval).squeeze()

# Gráfico de valores reais vs previstos
comparacao = pd.DataFrame({"Reais": yval, "Previstos": p_val})
fig2 = px.scatter(
    comparacao,
    x="Reais",
    y="Previstos",
    title="Valores Reais vs Previstos",
    labels={"x": "Valores Reais", "y": "Valores Previstos"}
)
fig2.show()

# Gráfico de resíduos
residuos = yval - p_val
fig3 = px.histogram(
    residuos,
    nbins=50,
    title="Distribuição dos Resíduos",
    labels={"value": "Resíduos", "count": "Frequência"}
)
fig3.show()


23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


## Exportação de Previsões

In [25]:
# Previsão para dados de teste
teste1 = teste_num.drop(["Id", "SalePrice"], axis=1)
teste = scaler.transform(imputer.transform(teste1))
p_teste = mdl.predict(teste).squeeze()

# Salvar previsões em CSV
resultados = pd.DataFrame({"Id": dados_teste["Id"], "SalePrice": p_teste})
resultados.to_csv("previsoes.csv", index=False)
print("Previsões salvas no arquivo previsoes.csv")


46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 643us/step
Previsões salvas no arquivo previsoes.csv
